## Build Enviroment

In [1]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 97.4MB 536kB/s ta 0:00:011  3% |█▎                              | 3.9MB 3.1MB/s eta 0:00:31    14% |████▋                           | 14.1MB 3.8MB/s eta 0:00:23    16% |█████▏                          | 15.8MB 7.6MB/s eta 0:00:11    22% |███████                         | 21.5MB 5.9MB/s eta 0:00:13    31% |██████████▏                     | 31.1MB 3.8MB/s eta 0:00:18
    100% |████████████████████████████████| 389kB 4.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/2c/4e/4f1ede0fd7a36278844a277f8d53c21f88f37f3754abf76a5d6224f76d4a/transformers-3.4.0-py3-none-any.whl
    100% |████████████████████████████████| 2.0MB 18.0MB/s ta 0:00:01   15% |████▉                           | 296kB 3.9MB/s eta 0:00:01
  Found existing installation: torch 1.2.0
    Uninstalling torch-1.2.0:
      Successfully uninstalled torch-1.2.0
  Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully 

# Train a model

To train a model you should specify the arguments, for example:

```bash
python run_metonymy_resolution.py \
--data_dir ../data \
--train_file conll_train.json \
--predict_file conll_test.json \
--output_dir ../output \
--do_train  \
--do_eval \
--do_mask
```

# You can also use our pretrained model

Download from, https://drive.google.com/file/d/1PCXkEFyK5OALQbF_64J6jSGO0IUjbYuf/view?usp=sharing

Unzip and put it to ``model_folder`` (you local path).

This is a pretrained bert-base-uncased model, use ./data/conll_train.json.

# Load pretrained model from ``model_folder``

In [1]:
import numpy as np
import sys
sys.path.append("./src")
from utils_metonymy import *

model_class = BertForWordClassification
tokenizer_class = BertTokenizer

In [2]:
model_folder = './output' # specify which dir the model been saved.

model = model_class.from_pretrained(model_folder)
tokenizer = tokenizer_class.from_pretrained(model_folder, do_lower_case=True)

# Input example

In [9]:
example = {'sentence': ['SOCCER', '-', 'ROMANIA', 'BEAT', 'LITHUANIA', 'IN', 'UNDER-21', 'MATCH.'],
           'pos': [2, 3]}

inputs = convert_single_example_to_input(example, tokenizer)

# Run the model on the input example

In [20]:
model.zero_grad()
model.eval()

logits = model(**inputs)[0]


preds = logits.detach().cpu().numpy()
preds = np.argmax(preds, axis=1)

label_map = {0:'literal',1:'metonymy'}

print(f'Target word:   {" ".join(example["sentence"][example["pos"][0]:example["pos"][1]])}')
print(f'Sentence:      {" ".join(example["sentence"])}')
print(f'Prediction:    {label_map[preds[0]]}')

Target word:   ROMANIA
Sentence:      SOCCER - ROMANIA BEAT LITHUANIA IN UNDER-21 MATCH.
Prediction:    literal
